In [2]:
import requests
import json
from bs4 import BeautifulSoup as BS
import time
import mysql.connector
import numpy as np
import re
import pandas as pd
import aws_config
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
cnx = mysql.connector.connect(
    host = aws_config.host,
    user = aws_config.user,
    password = aws_config.passwd)


# create cursor object
c = cnx.cursor()
cnx.database = 'movie_project'

In [ ]:
#c.execute("""CREATE database movie_project""")

In [4]:
url1 = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2000-01-01,2019-12-31&genres=comedy&sort=boxoffice_gross_us,desc&count=250&ref_=adv_prv'
url2 = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2000-01-01,2019-12-31&genres=comedy&sort=boxoffice_gross_us,desc&count=250&start=251&ref_=adv_nxt'
url3 = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2000-01-01,2019-12-31&genres=comedy&sort=boxoffice_gross_us,desc&count=250&start=501&ref_=adv_nxt'
url4 = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2000-01-01,2019-12-31&genres=comedy&sort=boxoffice_gross_us,desc&count=250&start=751&ref_=adv_nxt'
#response = requests.get(url)
all_urls = [url1, url2, url3, url4]

In [5]:
num_votes = []
box_office = []

for i in range(len(all_urls)):
    response = requests.get(all_urls[i])
    soup = BS(response.text, 'html.parser')
    num_votes_visible = soup.find_all('p', class_='sort-num_votes-visible')
    num_votes.append(num_votes_visible)
    


In [6]:
first = num_votes[0]
second = num_votes[1]
third = num_votes[2]
fourth = num_votes[3]

In [7]:
print(len(first))

250


In [8]:
new_cool_bucket = []
def box_office(number):
    for num in number:
        for x in range(0, 1000):
            try: 
                i = (num.find_all('span')[x])
                #ids.append(i['data-value'])
                new_cool_bucket.append(i['data-value'])
            except:
                continue

In [9]:
box_office(first)
box_office(second)
box_office(third)
box_office(fourth)


In [10]:
print(len(new_cool_bucket))

1999


In [11]:
box_office_gross = new_cool_bucket[1::2]

In [12]:
# new_bucket = []
# for i in range(len(new_cool_bucket)):
#      x = new_cool_bucket[i]
#     new_bucket.append(x.replace(",", ""))
#     print(new_bucket)

In [13]:
#box_office_gross

In [14]:
#function to get imdb id's

def get_url(urls):
    response = requests.get(urls)
    soup = BS(response.text, 'html.parser')
    urls = soup.find_all('div', class_='lister-list')
    return urls

In [15]:
#function to get imdb box office gross

def get_box_url(url):
    response = requests.get(url)
    soup = bs(response.text, 'html.parser')
    num_votes_vis = soup.find_all('p', class_='sort-num_votes-visble')
    return num_votes_vis

In [16]:
imdb_url1 = get_url(url1)
imdb_url2 = get_url(url2)
imdb_url3 = get_url(url3)
imdb_url4 = get_url(url4)

In [17]:
#Gets unique imdb id's from url

def get_movie_id(url_list):
    ids = []
    for result in url_list:
        for x in range(0, 250):
            try:
                i = (result.find_all('img')[x])
                ids.append(i['data-tconst'])
            except:
                continue
        return ids

In [18]:
imdb_ids1 = get_movie_id(imdb_url1)
imdb_ids2 = get_movie_id(imdb_url2)
imdb_ids3 = get_movie_id(imdb_url3)
imdb_ids4 = get_movie_id(imdb_url4)


In [19]:
#takes unique imdb id's and concatenates them with api key

def make_api_keys(ids):
    big_list = []
    for i in range(len(ids)):
        url = f'http://www.omdbapi.com/?i={ids[i]}&apikey=bb8ea380'
        big_list.append(url) 
    return big_list


In [20]:
imdb_api_keys1 = make_api_keys(imdb_ids1)
imdb_api_keys2 = make_api_keys(imdb_ids2)
imdb_api_keys3 = make_api_keys(imdb_ids3)
imdb_api_keys4 = make_api_keys(imdb_ids4)

In [21]:
def make_api_calls(api_keys):
    movie_data = []
    for i in range(0,250):
        url = api_keys[i]
        response = requests.get(url)
        data = response.json()
        movie_data.append(data)
    return movie_data
    

In [22]:
#makingi api calls
movies_1 = make_api_calls(imdb_api_keys1)
movies_2 = make_api_calls(imdb_api_keys2)
movies_3 = make_api_calls(imdb_api_keys3)
movies_4 = make_api_calls(imdb_api_keys4)

In [23]:
def movie_parser(movie_data):
    new_parsed_movies = []
    for i in range(len(movie_data)):
        try:
            new = (movie_data[i])
            #box_office = odd[i]
            [(v,k) for k, v in new.items()]
            imdb = new['Ratings'][0]
            im = (imdb['Value'])
            im_new = im.rpartition('/')[0]
            rotten_tomatoes = new['Ratings'][1]
            r = (rotten_tomatoes['Value'])
            r_new = r.rpartition('%')[0]
            metacritic = new['Ratings'][2]
            m = (metacritic['Value'])
            m_new = m.rpartition('/')[0]
            ratings = (new['Ratings'])
            runtime = new['Runtime']
            new_runtime = runtime.rpartition(' ')[0]
            new_genre = new['Genre']
            newer_genre = f'"{new_genre}"'
            new_tuple = (new['Title'], new['Year'], new_runtime, 
                            newer_genre, new['Director'], im_new, float(r_new), m_new, box_office_gross[i])
            new_parsed_movies.append(new_tuple)
        except IndexError:
             print("elements missing!!!!")  
    return new_parsed_movies

In [24]:
#parse desired values into tuples

parsed_1 = movie_parser(movies_1)
parsed_2 = movie_parser(movies_2)
parsed_3 = movie_parser(movies_3)
#parsed_4 = movie_parser(movies_4)

elements missing!!!!
elements missing!!!!
elements missing!!!!
elements missing!!!!
elements missing!!!!
elements missing!!!!
elements missing!!!!
elements missing!!!!
elements missing!!!!
elements missing!!!!
elements missing!!!!
elements missing!!!!
elements missing!!!!


In [26]:
create_table = """CREATE TABLE movie
(title varchar(100) PRIMARY KEY,
year int(11),
runtime int(11),
genre varchar(100),
director varchar(100),
imdb_rating float,
rotten_tomatoes int(5),
metacritic_rating int(11),
box_office varchar(100));
"""
c.execute(create_table)

In [27]:
def db_insert(cnx, c, parsed_results):
        stmnt = """
        INSERT INTO movie(title, year, runtime, genre, director, imdb_rating, rotten_tomatoes, metacritic_rating, box_office)
        VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s);
        """      
        c.executemany(stmnt, parsed_results)
        cnx.commit()

In [28]:
db_insert(cnx, c, parsed_1)
db_insert(cnx, c, parsed_2)
db_insert(cnx, c, parsed_3)